### To run this model yourself: go to Run -> Run All Cells in the top left menu bar.

In [1]:
%env PLATFORM_API_URL=https://studio.dev.epistemix.cloud/v1

env: PLATFORM_API_URL=https://studio.dev.epistemix.cloud/v1


In [ ]:
import epxexec
import epxexec.epxresults
from epxexec.fred_job import fred_job
from epxexec.visual.utils import default_plotly_template

import pandas as pd
import json
import plotly.express as px
import plotly.io as pio

pio.templates["epistemix"] = default_plotly_template()
pio.templates.default = "epistemix"

# Schelling Model

An implementation of the Schelling model of racial segregation in housing, using the household locations (N~8000) of Kewaunee County, Wisconsin. If you'd like to learn more about how this model works in detail, check out Lesson 8 in the Quickstart Guide.

In brief, agents are assigned one of two colors, in this case blue and red. At each timestep in the simulation, agents assess the color distribution of the residents in their block group and determine if enough agents are of a similar color to themselves.

If the agent is happy with the number of similar agents, they remain in their current household. If they are unhappy, the seek a new location to live from a shared list of available, empty houses.  This cycle repeats everyday.

## Key simulation variables

- number of households: 7,393
- fraction of household that are empty: 0.1
- desired neighborhood similarity fraction: 0.5

In [ ]:
%%time
schelling_job = fred_job("model/schelling_block.fred")

In [ ]:
# prep the data for visualization

block_history = schelling_job.runs[1].get_csv_output("block_color_history.csv")
house_info = schelling_job.runs[1].get_csv_output("household_info.csv")
agent_info = schelling_job.runs[1].get_csv_output("agent_info.csv")
agent_house_history = schelling_job.runs[1].get_csv_output("agent_house_history.csv")
agent_house_history = pd.merge(
    agent_house_history,
    house_info[["household_id","house_lat","house_long"]],
    how="left",on="household_id"
)

In the map below, each colored dot represents a householder agent who is either blue or red. Over time, the dots move around as agents who are unhappy seek out a new home in a block group that meets the similarity value defined the variable `desired_similarity`. The blue lines on the map show the boundaries of the block groups that are included in this simulation (these boundaries were downloaded from the [U.S. census website](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.2010.html#list-tab-OZSS213R7R9GVKFO5Y).)

This animated map shows that by about 14 days into the simulation, the population has reached a stable equilibrium situation characterized by completely homogenous block groups - all despite the fact that the population started with a roughly 50:50 spatial distribution of color and that people are happy living with up to 50% different colored agents.

In [ ]:
# calculate lat, long to pass to mapbox for map center
lat_cen = house_info['house_lat'].median()
long_cen = house_info['house_long'].median()

# set up Epistemix house map tiles
mapstyle="mapbox://styles/pnowell/cl4n9fic8001i15mnfmozrt8j"
token="pk.eyJ1IjoicG5vd2VsbCIsImEiOiJja201bHptMXkwZnQyMnZxcnFveTVhM2tyIn0.Pyarp9gHCON4reKvM2fZZg"

# load the block group boundary file
f = open("Kewaunee_County_WI_blockgroups.geojson")
blocks_json = json.load(f)

# create the figure
fig1 = px.scatter_mapbox(
    agent_house_history, lat="house_lat", lon="house_long",
    color="agent_color",
    color_continuous_scale=[
                (0,"rgba(235,90,54,1)"), # red
                (1,"rgba(40, 95, 223, 1)") # blue
    ],
    animation_frame='date', height=700, zoom=9.25,
    labels={'agent_color':'Agent color'}
)

fig1.update_layout(
    mapbox={'layers': [{
        'source': blocks_json,
        'type': "line", 'below': "traces", 'color': "#6383F3",
        'opacity': 1.0
    }]}
)

fig1.update_layout(mapbox_style=mapstyle, mapbox_accesstoken=token)
fig1.update(layout_coloraxis_showscale=False)
fig1.update_layout(margin={"r":20,"t":10,"l":10,"b":3})
fig1.show()

We can also examine the aggregate characteristics of the county's block groups. The map below shows each block group of Kewaunee County, with the color representing the fraction of households that are blue.

In [ ]:
# calculate lat, long to pass to mapbox for map center
lat_cen = house_info['house_lat'].median()
long_cen = house_info['house_long'].median()

# set up Epistemix house map tiles
mapstyle="mapbox://styles/pnowell/cl4n9fic8001i15mnfmozrt8j"
token="pk.eyJ1IjoicG5vd2VsbCIsImEiOiJja201bHptMXkwZnQyMnZxcnFveTVhM2tyIn0.Pyarp9gHCON4reKvM2fZZg"

# load the block group boundary file
f = open("Kewaunee_County_WI_blockgroups.geojson")
blocks_json = json.load(f)

# create the figure
fig2 = px.choropleth_mapbox(
    block_history, geojson=blocks_json,
    locations='blockgroup_id', color='frac_blue',
    color_continuous_scale=[
        (0,"rgba(235,90,54,1)"), # blue
        (1,"rgba(40, 95, 223, 1)") # red
        #(0,"rgba(249, 183, 45,1)"),
        #(1,"rgba(196, 200, 204,1)"),
        #(1,"rgba(104, 148, 255,1)")
    ],
    range_color=(0, 1),featureidkey="properties.GEOID10",
    hover_data=block_history.columns,
    animation_frame='date',
    zoom=9.25, height=700,
    center = {"lat": lat_cen,"lon": long_cen},
    labels={'frac_blue':'Blue fraction'}
)

fig2.update_layout(mapbox_style=mapstyle, mapbox_accesstoken=token)
fig2.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 300
fig2.update_layout(coloraxis_colorbar_orientation='h')
fig2.update_layout(coloraxis_colorbar_y=-0.1)
fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0.3})
fig2.show()